In [1]:
import numpy as np
import os
import cv2
import joblib
from sklearn.preprocessing import StandardScaler
from skimage import feature
from imutils import paths
from imutils import build_montages
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


In [2]:
# Extracting numeric data from image
def quantify_image(image):
    features = feature.hog(
        image,
        orientations=12,
        pixels_per_cell=(8, 8),
        cells_per_block=(2, 2),
        transform_sqrt=True,
        block_norm="L2-Hys",
    )
    return features


In [3]:
#testing quantify_image
# Create a dummy image and test feature extraction
test_img = np.zeros((200, 200), dtype="uint8")
cv2.circle(test_img, (100, 100), 50, 255, 2)

hog_features = quantify_image(test_img)

print("Feature vector length:", len(hog_features))

Feature vector length: 27648


In [4]:
#loading data from folder
def load_split(dataset_path):
    image_paths = list(paths.list_images(dataset_path))
    data = []
    labels = []

    for image_path in image_paths:
        # Label is the folder name (healthy / parkinson)
        label = image_path.split(os.path.sep)[-2]


        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (200, 200))

        image = cv2.threshold(image, 0, 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
        features = quantify_image(image)

        data.append(features)
        labels.append(label)

    return np.array(data), np.array(labels)

In [5]:
#loading testing and training data for spiral
trainX_spiral, trainY_spiral = load_split("dataset_handwriting/training/spiral")
testX_spiral, testY_spiral = load_split("dataset_handwriting/testing/spiral")

scaler = StandardScaler()
trainX_spiral = scaler.fit_transform(trainX_spiral)
testX_spiral = scaler.transform(testX_spiral)


print("Number of samples:", trainX_spiral.shape[0])
print("Feature length:", trainX_spiral.shape[1])
print("Labels found:", np.unique(trainY_spiral))

ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
df = pd.DataFrame(trainX_spiral)
df['label'] = trainY_spiral

print("Dataset shape:", df.shape)
print("\nClass distribution:")
print(df['label'].value_counts())

print("\nBasic statistics (first 5 features):")
print(df.iloc[:, :5].describe())

In [7]:
le = LabelEncoder()
le.fit(["healthy", "parkinson"])   # force correct order
trainY_enc = le.transform(trainY_spiral)

# Train the model
model = SVC(
    kernel="rbf",
    C=10,
    gamma="scale",
    class_weight="balanced",
    probability=True,
    random_state=42,
)
model.fit(trainX_spiral, trainY_enc)


ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
print("Classes:", le.classes_)

Classes: ['healthy' 'parkinson']


In [ ]:
# Encode test labels
testY_enc = le.transform(testY_spiral)

# Make predictions
predictions = model.predict(testX_spiral)

# Confusion matrix
cm = confusion_matrix(testY_enc, predictions)
tn, fp, fn, tp = cm.ravel()

# Metrics
accuracy = (tp + tn) / cm.sum()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
error_rate = 1 - accuracy

print("Confusion Matrix:\n", cm)
print("Accuracy:", accuracy)
print("Sensitivity:", sensitivity)
print("Specificity:", specificity)
print("Error rate:", error_rate)


Confusion Matrix:
 [[ 8  7]
 [ 1 14]]
Accuracy: 0.7333333333333333
Sensitivity: 0.9333333333333333
Specificity: 0.5333333333333333
Error rate: 0.2666666666666667


In [ ]:
print("SVC Model:")
accuracy = (tp + tn) / cm.sum()
print("Accuracy:", accuracy)

SVC Model:


NameError: name 'tp' is not defined

In [ ]:
#dumping the model
#joblib.dump(model, "parkinsons_spiral_model.pkl")
#joblib.dump(le, "label_encoder_spiral.pkl")
#joblib.dump(scaler, "scaler_spiral.pkl")

In [ ]:
#building CNN model

def load_images(dataset_path, img_size=128):
    data = []
    labels = []

    for label in os.listdir(dataset_path):
        label_path = os.path.join(dataset_path, label)

        for img_name in os.listdir(label_path):
            img_path = os.path.join(label_path, img_name)

            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image, (img_size, img_size))
            image = image / 255.0  # normalize

            data.append(image)
            labels.append(1 if label == "parkinson" else 0)

    data = np.array(data).reshape(-1, img_size, img_size, 1)
    labels = np.array(labels)

    return data, labels

In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation="relu", input_shape=(128,128,1)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation="relu"),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation="relu"),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.5),

    Dense(1, activation="sigmoid")  # Binary classification
])


c:\Users\ayush\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,193 (12.60 MB)

 Trainable params: 3,304,193 (12.60 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
#loading data
def load_images(dataset_path, img_size=128):
    data = []
    labels = []

    for label in os.listdir(dataset_path):
        folder = os.path.join(dataset_path, label)

        for img_name in os.listdir(folder):
            img_path = os.path.join(folder, img_name)

            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image, (img_size, img_size))
            image = image / 255.0

            data.append(image)
            labels.append(1 if label == "parkinson" else 0)

    data = np.array(data).reshape(-1, img_size, img_size, 1)
    labels = np.array(labels)

    return data, labels

trainX_spiral, trainY_spiral = load_images("dataset_handwriting/training/spiral")
testX_spiral, testY_spiral = load_images("dataset_handwriting/testing/spiral")


In [ ]:

history = model.fit(
    trainX_spiral, trainY_spiral,
    validation_data=(testX_spiral, testY_spiral),
    epochs=20,
    batch_size=16
)

Epoch 1/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 175ms/step - accuracy: 0.5142 - loss: 0.6891 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 2/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 158ms/step - accuracy: 0.5252 - loss: 0.6924 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 3/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 157ms/step - accuracy: 0.6661 - loss: 0.6884 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 4/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 157ms/step - accuracy: 0.5116 - loss: 0.6916 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 5/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 173ms/step - accuracy: 0.4980 - loss: 0.6941 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 6/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 160ms/step - accuracy: 0.4702 - loss: 0.6962 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 7/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 166ms/step - accuracy: 0.5237 - loss: 0.6909 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 8/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 154ms/step - accuracy: 0.5654 - loss: 0.6938 - val_accuracy: 0.5000 - val_loss:

In [ ]:
print("CNN Model:")
print("Final training accuracy:",
      history.history["accuracy"][-1])

print("Final validation accuracy:",
      history.history["val_accuracy"][-1])


CNN Model:
Final training accuracy: 0.5555555820465088
Final validation accuracy: 0.5
